# Определение тематики новости с помощью простой нейросети
## Описание задачи
1. Извлечение текста из PDF-документов:
   - Используя библиотеку PyPDF2, pdfminer или PyMuPDF, реализовать функцию, которая принимает на вход путь к PDF-файлу и возвращает извлеченный текст.

2. Предобработка текста:
   - Реализовать функцию для предобработки извлеченного текста. Включить в нее следующие шаги:
     - Удаление специальных символов и лишних пробелов.
     - Токенизация текста (разделение на слова).

3. Обучение модели:
   - С использованием библиотеки Keras или PyTorch создать простую нейронную сеть для классификации текста. Для обучения использовать набор данных, содержащий тексты и соответствующие метки (например, категории документов). В работе будет использован датасет 20 Newsgroups.

4. Интерфейс для пользователя:
   - Реализовать простой интерфейс командной строки или веб-интерфейс с использованием Flask, который позволит пользователю загрузить PDF-документ и получить предсказанную категорию документа.

5. Документация:
   - Подготовить к проекту документацию, описывающую структуру кода, инструкции по установке и запуску, а также примеры использования.

Дополнительные требования:
- Используйте Git для контроля версий вашего проекта.
- Напишите тесты для ключевых функций вашего кода.
- Обратите внимание на обработку ошибок (например, что происходит при загрузке некорректного PDF).

## Import dependencies

In [1]:
import numpy as np
import fitz
import re
import nltk
import joblib
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# Функция для выделения текста из .pdf
def extract_text(path):
    text = ""
    with fitz.open(path) as doc:
        for page in doc:
            text += page.get_text()
    return text

In [3]:
# Функция для токенизации
def tokenize_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    tokens = nltk.word_tokenize(text.lower(), language='english')
    return ' '.join(tokens)

In [4]:
# Загрузка и подготовка данных
data = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
texts = data['data']
labels = data['target']

# Очистка текста от лишних символов
texts = [tokenize_text(text) for text in texts]

# Разделение на выборки
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42, stratify=labels)


# Векторизация выборок
vectorizer = TfidfVectorizer(max_features=15000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [5]:
# Обучаем модель
model = keras.Sequential([
    keras.layers.Input(shape=(X_train_vec.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(
    X_train_vec.toarray(), y_train,
    epochs=30, validation_data=(X_test_vec.toarray(), y_test),
    callbacks=EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
)

Epoch 1/30
472/472 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.3928 - loss: 2.6827 - val_accuracy: 0.7066 - val_loss: 1.5091
Epoch 2/30
472/472 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7764 - loss: 1.1960 - val_accuracy: 0.7369 - val_loss: 1.0384
Epoch 3/30
472/472 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8553 - loss: 0.6922 - val_accuracy: 0.7451 - val_loss: 0.9123
Epoch 4/30
472/472 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9109 - loss: 0.4629 - val_accuracy: 0.7541 - val_loss: 0.8659
Epoch 5/30
472/472 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9361 - loss: 0.3262 - val_accuracy: 0.7565 - val_loss: 0.8517
Epoch 6/30
472/472 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9533 - loss: 0.2465 - val_accuracy: 0.7517 - val_loss: 0.8554
Epoch 7/30
472/472 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9648 - loss: 0.1901 - val_accuracy: 0.7480 - val_loss: 0.8614
Epoch 8/30
472/472 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9685 - loss: 0.1567 - val_accuracy: 0.

### Итоги обучения модели

#### Финальная модель
Простая нейросеть с одним скрытым слоем обладает высокой метрикой выше 0.9 на тренировочной выборке и относительно невысокой метрикой 0.75 на тестовой выборке, что говорит о переобучении модели.

#### Борьба с переобучением
Основная сложность при обучении модели в заданных условиях - переобучение. Возможно, это связано с выбранным методом векторизации текстов (TF-IDF может быть достаточно грубым, так как не предусматривает логической связи между словами). 

В финальной версии модели для борьбы с переобучением была использована регуляризация - Dropout и ранняя остановка обучения при 3 эпохах без улучшения accuracy. 

В предыдущих версиях вручную были проверены:
- модели с более сложной архитектурой (512-256-128-64; 256-128-64;128-64 нейронов - accuracy не выше 0.7 на валидационной выборке, из-за быстрого переобучения) в комбинации с L2 регуляризацией и Dropout(0.1-0.7);
- модели с финальной архитектурой в комбинации с L2 регуляризацией и Dropout (accuracy не выше 0.73 на валидационной выборке);
- векторизаторы с разными ограничениями на максимальное количество признаков (5000, 7000, 10000, 12000, 15000, 20000), в том числе с биграммами;
- модели с меньшим количеством нейронов в скрытых слоях плохо предсказывают тексты и сильно переобучаются (accuracy не выше 0.63-0.65 из-за быстрого переобучения);

В результате было принято решение остановиться на архитектуре с одним скрытым слоем и 64 нейронами, так как такая модель показала лучший результат на валидационной выборке - 0.75.

#### Потенциальные точки для улучшения модели
Модель, использующая данные, обработанные векторизатором BERT, может показать результат лучше, в сравнении с TF-IDF, так как BERT способна уловить семантический смысл предложений и словосочетаний.

### Анализ ошибок модели
Изучим основные метрики для каждого класса

In [6]:
# Предсказываем вероятности
y_test_predicted_probas = model.predict(X_test_vec.toarray())
# Назначаем классы в соответствии с вероятностями
y_test_predicted = np.argmax(y_test_predicted_probas,axis=1)


# Строим итоговую таблицу с метриками
print(classification_report(y_test, y_test_predicted, target_names=data.target_names))

118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 807us/step
                          precision    recall  f1-score   support

             alt.atheism       0.67      0.64      0.65       160
           comp.graphics       0.77      0.75      0.76       195
 comp.os.ms-windows.misc       0.71      0.69      0.70       197
comp.sys.ibm.pc.hardware       0.67      0.67      0.67       196
   comp.sys.mac.hardware       0.76      0.74      0.75       193
          comp.windows.x       0.87      0.86      0.86       198
            misc.forsale       0.82      0.75      0.78       195
               rec.autos       0.51      0.84      0.63       198
         rec.motorcycles       0.84      0.68      0.75       199
      rec.sport.baseball       0.88      0.84      0.86       199
        rec.sport.hockey       0.95      0.89      0.91       200
               sci.crypt       0.86      0.81      0.84       198
         sci.electronics       0.71      0.79      0.75       197
                 sci.med       0

Некоторые редкие темы обладают относительно низкой recall и f1, что влияет в том числе и на общую accuracy: например, talk.religion.misc и alt.atheism. Так как таких текстов меньше чем обычно и их метрики не нулевые, можем считать, что модель не игнорирует редкие классы. Для улучшения можно дополнить датасет новостями на эти темы, особенно talk.religion.misc.

У основных и наиболее распространенных тем: rec.sport.hockey, comp.windows.x, и других, метрики recall и f1 выше 0.85, это отличный результат.

В целом, общий результат работы модели с учетом использования простой неросети и TF-IDF можно считать отличным.

## Технический блок проекта

In [7]:
# Сохраняем предобученный векторизатор и модель
joblib.dump(vectorizer, 'my_vectorizer.pkl')
model.save('my_model.keras')


# Нам понадобится файл с сохраненными категориями, поэтому соберем его
categories = data.target_names
with open('categories.json', 'w', encoding='utf-8') as f:
    json.dump(categories, f, ensure_ascii=False, indent=2)